# 数值分析 实验3
2019011265 计93 丁韶峰

In [36]:
import numpy as np
import matplotlib.pyplot as plt

Cholesky分解

In [37]:
def Cholesky(A, n):
  L = np.zeros_like(A)

  for j in range(n):
    L[j][j] = A[j][j]
    for k in range(j):
      L[j][j] -= L[j][k] ** 2
    L[j][j] = np.sqrt(L[j][j])
    for i in range(j + 1, n):
      L[i][j] = A[i][j]
      for k in range(j):
        L[i][j] -= L[i][k] * L[j][k]
      L[i][j] /= L[j][j]

  return L

In [38]:
def solve(L, b, n):
  # Ly = b
  y = np.zeros_like(b)
  for i in range(n):
    y[i] = b[i]
    for j in range(0, i):
      y[i] -= L[i][j] * y[j]
    y[i] /= L[i][i]

  # LTx = y
  x = np.zeros_like(b)
  for i in reversed(range(n)):
    x[i] = y[i]
    for j in reversed((range(i + 1, n))):
      x[i] -= L[j][i] * x[j]
    x[i] /= L[i][i]
  return x

In [39]:
def compute(n):
  H = np.fromfunction(lambda i, j : 1.0 / (i + j + 1), (n, n))
  ones = np.ones(n)
  print("cond is {}".format(np.linalg.cond(H)))
  b = np.dot(H, ones)
  L = Cholesky(H, n)
  x = solve(L, b, n)
  r = np.max(np.abs(b - np.dot(H, x)))
  delta = np.max(np.abs(ones - x))
  print("no disturbance, r is {}, delta is {}".format(r, delta))
  x = solve(L, b + np.random.normal(0, 1e-7, n), n)
  r = np.max(np.abs(b - np.dot(H, x)))
  delta = np.max(np.abs(ones - x))
  print("with disturbance, r is {}, delta is {}".format(r, delta))

In [40]:
n_list = [8, 10, 12]
for n in n_list:
  compute(n)

cond is 15257575566.627958
no disturbance, r is 4.440892098500626e-16, delta is 3.2588079057482844e-07
with disturbance, r is 1.2478448052632984e-07, delta is 44.92419836754452
cond is 16025028168113.176
no disturbance, r is 8.881784197001252e-16, delta is 0.0005932324447111004
with disturbance, r is 1.823562434388748e-07, delta is 886386.3676761363
cond is 1.6211639047474996e+16
no disturbance, r is 2.220446049250313e-16, delta is 0.47735610905468484
with disturbance, r is 1.9918960525444618e-07, delta is 573327055.2049657
